In [ ]:
def kruskal_h_test(df,target_col,col):
    
    h_statistic, p_value = stats.kruskal(*[group[target_col] for name, group in df.groupby(col)])
    
    results_df = pd.DataFrame({
        'categorical_column': [col],
        'p_value': [p_value]
    })
    
    return results_df


def error_rate_table(df,target_bucket,col,col_name,x,y,order_flag,ord_list,search_filter):
    
    
    total_searches = df.groupby(col)['createtripid_region'].count().rename('total_searches')
    
    
    total_errors = df.groupby(col)[target_bucket].sum().rename('target_errors')
    
    
    searches_errors_df = pd.concat([total_searches, total_errors], axis=1)

    searches_errors_df['normalized_error_rate'] = searches_errors_df['target_errors'] / searches_errors_df['total_searches']
    searches_errors_df = searches_errors_df.sort_values('normalized_error_rate', ascending=False)

    searches_mean = searches_errors_df['total_searches'].mean()

    if search_filter == True: # filter by search mean for true error rate values
        searches_errors_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0) & (searches_errors_df['total_searches'] >= searches_mean)].sort_values('normalized_error_rate', ascending=False)
    else:
        searches_errors_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0)].sort_values('normalized_error_rate', ascending=False)

    # only show top 25 rows by error rate
    searches_errors_df = searches_errors_df[:25]

    searches_errors_df['searches_mean'] = searches_mean

    groups = searches_errors_df.reset_index()[:5][col]

    return groups

def error_rate_table_all(df,target_bucket,col,col_name,x,y,order_flag,ord_list,search_filter):
    
    
    total_searches = df.groupby(col)['createtripid_region'].count().rename('total_searches')
    
    total_errors = df.groupby(col)[target_bucket].sum().rename('target_errors')
    
    searches_errors_df = pd.concat([total_searches, total_errors], axis=1)

    
    searches_errors_df['normalized_error_rate'] = searches_errors_df['target_errors'] / searches_errors_df['total_searches']
    searches_errors_df = searches_errors_df.sort_values('normalized_error_rate', ascending=False)

    searches_mean = searches_errors_df['total_searches'].mean()

    if search_filter == True: # filter by search mean for true error rate values
        searches_errors_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0) & (searches_errors_df['total_searches'] >= searches_mean)].sort_values('normalized_error_rate', ascending=False)
    else:
        searches_errors_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0)].sort_values('normalized_error_rate', ascending=False)

    # only show top 25 rows by error rate
    searches_errors_df = searches_errors_df[:25]

    searches_errors_df['searches_mean'] = searches_mean


    return searches_errors_df


def error_rate(df,target_bucket,col,col_name,x,y,order_flag,ord_list,search_filter):
    
    total_searches = df.groupby(col)['createtripid_region'].count().rename('total_searches')
    
    total_errors = df.groupby(col)[target_bucket].sum().rename('target_errors')
    
    searches_errors_df = pd.concat([total_searches, total_errors], axis=1)

    
    searches_errors_df['normalized_error_rate'] = searches_errors_df['target_errors'] / searches_errors_df['total_searches']
    searches_errors_df = searches_errors_df.sort_values('normalized_error_rate', ascending=False)

    searches_mean = searches_errors_df['total_searches'].mean()

    if search_filter == True: # filter by search mean for true error rate values
        searches_errors_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0) & (searches_errors_df['total_searches'] >= searches_mean)].sort_values('normalized_error_rate', ascending=False)
    else:
        searches_errors_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0)].sort_values('normalized_error_rate', ascending=False)

    # only show top 25 rows by error rate
    searches_errors_df = searches_errors_df[:25]

    error_data = searches_errors_df['normalized_error_rate'].to_dict()
    
    def custom_sort_key(item):
        key, value = item
        return ord_list.index(key)
        
    if order_flag == True:
        error_data = dict(sorted(error_data.items(), key=custom_sort_key, reverse=False))

    # Creating a horizontal bar chart
    plt.figure(figsize=(x,y))
    plt.barh(list(error_data.keys()), list(error_data.values()), color=plt.cm.viridis(np.linspace(0, 1, len(df))))
    plt.xlabel(target_bucket + ' rate')
    plt.ylabel(col_name)
    plt.title(target_bucket+' rate by '+col_name)
    plt.gca().invert_yaxis()  
    plt.show()

    searches_errors_df['searches_mean'] = searches_mean

    return searches_errors_df

def stacked_heatmap(df,target_buckets,col,col_name,x,y,flag,order_flag,ord_list):

    all_cols = [col]+target_buckets
    
    new_df = df[[col]].drop_duplicates()
    
    for target_bucket in target_buckets:
        total_searches = df.groupby(col, observed=False)['createtripid_region'].count().rename('total_searches')
        
        total_errors = df.groupby(col, observed=False)[target_bucket].sum().rename('target_errors')
        
        grouped_data = pd.concat([total_searches, total_errors], axis=1)
        
        grouped_data[target_bucket + '_rate'] = grouped_data['target_errors'] / grouped_data['total_searches']
        
        new_df = pd.merge(new_df, grouped_data[[target_bucket + '_rate']],
                          left_on=col, right_index=True, suffixes=('', '_'+target_bucket))
        new_df = new_df.reset_index(drop=True)
    
    
    new_df[col] = pd.Categorical(new_df[col], categories=ord_list, ordered=True)
    
    new_df = new_df.sort_values(by=col)
    
    new_df.set_index(col, inplace=True)

    plt.figure(figsize=(8,6))
    sns.heatmap(new_df, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.xlabel(col)
    plt.ylabel('Error Types')
    plt.title('Error Rate Heatmap')
    plt.show()

    return new_df

def grouped_error_rate_3(target_bucket,cols,col_names,x,y):
    total_searches = df.groupby(cols)['createtripid_region'].count().rename('total_searches')
    
    total_errors = df.groupby(cols)[target_bucket].sum().rename('target_errors')
    
    searches_errors_df = pd.concat([total_searches, total_errors], axis=1)
    
    searches_errors_df['normalized_error_rate'] = searches_errors_df['target_errors'] / searches_errors_df['total_searches']
  
    searches_mean = searches_errors_df['total_searches'].mean()
    
    filtered_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0) & (searches_errors_df['total_searches'] >= searches_mean)].sort_values('normalized_error_rate', ascending=False)

    filtered_df['searches_mean'] = searches_mean
    
    filtered_df_reset = filtered_df.reset_index()
    
    filtered_df_reset['group_label'] = filtered_df_reset.apply(lambda row: f"{row[cols[0]]}, {row[cols[1]]}, {row[cols[2]]}", axis=1)

    filtered_df_reset = filtered_df_reset[:25]
    
    plt.figure(figsize=(x,y))
    sns.barplot(x='normalized_error_rate', y='group_label', data=filtered_df_reset, palette="viridis")
    
    plt.xlabel(target_bucket)
    plt.ylabel(col_names)
    plt.title(target_bucket+' Error Rate by '+col_names)
    plt.tight_layout()
    
    plt.show()

    return filtered_df_reset

def grouped_error_rate_2(df,target_bucket,cols,col_names,x,y):
    total_searches = df.groupby(cols)['createtripid_region'].count().rename('total_searches')
    
    total_errors = df.groupby(cols)[target_bucket].sum().rename('target_errors')
    
    searches_errors_df = pd.concat([total_searches, total_errors], axis=1)
    
    searches_errors_df['normalized_error_rate'] = searches_errors_df['target_errors'] / searches_errors_df['total_searches']
    
    searches_mean = searches_errors_df['total_searches'].mean()
    
    filtered_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0) & (searches_errors_df['total_searches'] >= searches_mean)].sort_values('normalized_error_rate', ascending=False)

    filtered_df['searches_mean'] = searches_mean
    
    filtered_df_reset = filtered_df.reset_index()
    
    filtered_df_reset['group_label'] = filtered_df_reset.apply(lambda row: f"{row[cols[0]]}, {row[cols[1]]}", axis=1)

    filtered_df_reset = filtered_df_reset[:25]
    
    plt.figure(figsize=(x,y))
    sns.barplot(x='normalized_error_rate', y='group_label', data=filtered_df_reset, palette="viridis")
    
    plt.xlabel(target_bucket)
    plt.ylabel(col_names)
    plt.title(target_bucket+' Error Rate by '+col_names)
    plt.tight_layout()
    
    plt.show()

    return filtered_df_reset


def grouped_error_rate_2_table(df,target_bucket,cols,col_names,x,y):
    total_searches = df.groupby(cols)['createtripid_region'].count().rename('total_searches')
    
    total_errors = df.groupby(cols)[target_bucket].sum().rename('target_errors')
    
    searches_errors_df = pd.concat([total_searches, total_errors], axis=1)
    
    searches_errors_df['normalized_error_rate'] = searches_errors_df['target_errors'] / searches_errors_df['total_searches']
    
    searches_mean = searches_errors_df['total_searches'].mean()
    
    filtered_df = searches_errors_df[(searches_errors_df['normalized_error_rate'] > 0) & (searches_errors_df['total_searches'] >= searches_mean)].sort_values('normalized_error_rate', ascending=False)

    filtered_df['searches_mean'] = searches_mean
    
    filtered_df_reset = filtered_df.reset_index()
    
    filtered_df_reset['group_label'] = filtered_df_reset.apply(lambda row: f"{row[cols[0]]}, {row[cols[1]]}", axis=1)

    filtered_df_reset = filtered_df_reset[:25]

    return filtered_df_reset


def heatmap2(df_tmp,filtered_df_reset,col1,col2,target_bucket,order):
    df_tmp = df_tmp[['createtripid_region',col1,col2,target_bucket]]
    filtered_df_reset = filtered_df_reset.rename(columns={'group_label':col1})
    
    filtered_df_reset = filtered_df_reset[:5][[col1]]
    considered_group = pd.merge(filtered_df_reset,df_tmp,on=col1)
    
    total_searches = considered_group.groupby([col1,col2],observed=True)['createtripid_region'].count().rename('total_searches')
    
    total_errors = considered_group.groupby([col1,col2])[target_bucket].sum().rename('target_errors')
    
    grouped_data = pd.concat([total_searches, total_errors], axis=1)
    
    grouped_data[target_bucket + '_rate'] = grouped_data['target_errors'] / grouped_data['total_searches']
    
    grouped_data = grouped_data.reset_index()
    
    grouped_data[col2] = pd.Categorical(grouped_data[col2], categories=order, ordered=True)

    pivot_df = grouped_data.pivot_table(values=target_bucket + '_rate',
                              index=col2,
                              columns=col1,
                              aggfunc='sum', fill_value=0)
    pivot_df = pivot_df.T
    # Create the heatmap
    plt.figure(figsize=(10, 4))  # Adjust the figure size as needed
    sns.heatmap(pivot_df, cmap='viridis', annot=True, fmt='.2f', linewidths=0.5)
    
    plt.xlabel(col2)
    plt.ylabel(col1)
    plt.title(target_bucket+' rate Heatmap')
    plt.tight_layout()
    
    plt.show()

    return pivot_df


# converting children ages column: -1 for no child and mean of 1 or more children ages 
def convert_empty_brackets_and_mean(value):
    if value == '{}':
        return 'no child'#-1
    else:
        set_value = eval(value)  # Convert the string to a set
        if len(set_value) > 0:
            return sum(set_value) / len(set_value)
        else:
            return 'no child'#-1


def column_conversions(df):
    
    df['success'] = df['success'].replace({True: 1, False: 0})
    
    # converting array_to_string column to theme preference columns (one hot encoded)
    array_to_string_df = (df['array_to_string'].str.split(r'\s*,\s*', expand=True)
       .apply(pd.Series.value_counts, 1)
       .iloc[:, 1:]
       .fillna(0, downcast='infer'))
    
    df = pd.concat([df, array_to_string_df], axis=1)
    
    df.drop(columns=['array_to_string'], inplace=True)
    
    
    df['childages'] = df['childages'].apply(convert_empty_brackets_and_mean)
    df['childages'] = df['childages'].astype(str)
    
    # # temporal columns
    
    df['request_weekday'] = df['requestdate'].dt.weekday
    df['request_hour'] = df['requestdate'].dt.hour.astype(str)
    
    df['creation_weekday'] = df['creationdate'].dt.weekday
    df['creation_hour'] = df['creationdate'].dt.hour.astype(str)
    
    # Extract year, quarter, month, and weekday from 'preferreddate'
    df['preferred_year'] = df['preferreddate'].dt.year.astype(str)
    df['preferred_quarter'] = df['preferreddate'].dt.quarter.astype(str)
    df['preferred_month'] = df['preferreddate'].dt.month
    df['preferred_weekday'] = df['preferreddate'].dt.weekday
    
    # converting date columns to day names
    df['request_weekday'] = df['request_weekday'].apply(lambda x: calendar.day_name[x])
    df['creation_weekday'] = df['creation_weekday'].apply(lambda x: calendar.day_name[x])
    df['preferred_weekday'] = df['preferred_weekday'].apply(lambda x: calendar.day_name[x])
    
    # converting preferreddate column to month name 
    df['preferred_month'] = df['preferred_month'].astype(int).apply(lambda x: calendar.month_name[x])
    
    df['days_between'] = (df['preferreddate'] - df['requestdate']).dt.days

    return df

def bucketizing_columns(df):
    df['request_hour'] = df['request_hour'].astype(int)
    df['creation_hour'] = df['creation_hour'].astype(int)
    df['childages'] = df['childages'].replace('no child',-1)
    df['childages'] = df['childages'].astype(float)
    
    # # creation_hour
    df.loc[df['childages'] ==-1, 'child_ages_bucket'] = 'no child'
    df.loc[(df['childages'] > 0)&(df['childages'] <= 6), 'child_ages_bucket'] = 'child age 0 to 6'
    df.loc[(df['childages'] > 6)&(df['childages'] <= 10), 'child_ages_bucket'] = 'child age 7 to 10'
    df.loc[(df['childages'] > 10), 'child_ages_bucket'] = 'child age 10+'
    
    bins = [2, 6, 7, 11, 18, 25]  # Defining the bin edges based on the logical bins 
    labels = ['Short Trips', 'Medium Short Trips', 'Medium Trips', 'Medium Long Trips', 'Long Trips']  # Bin labels
    
    df['trip_duration_bucket'] = pd.cut(df['durationdays'], bins=bins, labels=labels, right=True)
    
    
    df['failtime_bucket'] = pd.cut(df['failtime'],
                                        bins=[-float('inf'), 0, 5, 10, float('inf')],
                                        labels=['NaN', '1 to 5 seconds', '6 to 10 seconds', '10+ seconds'])
    
    df['tripbuildtime_bucket'] = pd.cut(df['tripbuildtime'],
                                        bins=[-float('inf'),0, 5, 15, 30,60,float('inf')],
                                        labels=['NaN','buildtime 0 to 5', 'buildtime 6 to 15', 'buildtime 16 to 30','buildtime 31 to 60', 'buildtime >60'])
    
    
    df['request_hour_bucket'] = pd.cut(df['request_hour'],
                                        bins=[0,5, 10, 15, 24],
                                        labels=['12am to 5am', '5am to 10am', '10am to 3pm','3pm to 12am'],right=False, include_lowest=True)
    
    df['creation_hour_bucket'] = pd.cut(df['creation_hour'],
                                        bins=[0,5, 10, 15, 24],
                                        labels=['12am to 5am', '5am to 10am', '10am to 3pm','3pm to 12am'],right=False, include_lowest=True)
    
    
    bins = [7, 37, 58, 132, 193, 350, 2979] 
    labels = ['Very Short', 'Short to Medium', 'Medium',
              'Medium to Long', 'Long', 'Very Long']
    
    df['lead_time_category'] = pd.cut(df['days_between'], bins=bins, labels=labels, right=False)
    
    # converting below columns to string columns (for analysis against errors)
    
    to_str_cols = ['adultcount', 'childcount', 'roomcount',
     'Arts and Culture', 'Food and Drink', 'Good for kids', 'History', 'NA', 'Nature', 'Nightlife', 'Relaxation', 'preferred_year', 'preferred_quarter',
     'preferred_month']
    
    for col in to_str_cols:
        df[col] = df[col].astype(str)
    
    categ_cols = ['region','closestcity','twinroompreferred',
                 'Arts and Culture', 'Food and Drink', 'Good for kids', 'History', 'NA',
           'Nature', 'Nightlife', 'Relaxation','adultcount','childcount','roomcount',
    'request_weekday','request_hour','creation_weekday','creation_hour','preferred_year','preferred_quarter','preferred_month',
    'preferred_weekday','child_ages_bucket','trip_duration_bucket','tripbuildtime_bucket','lead_time_category']
    
    # 'not considered : failtime_bucket'
    
    categorical_cols = to_str_cols + categ_cols
    categorical_cols = list(set(categorical_cols))

    return df,categorical_cols

def combined_kruskal_function(df,columns_to_analyze,errors_rearranged):
    
    combined_kruskal_test = pd.DataFrame(columns=['categorical_column'])
    
    for target_col in errors_rearranged:
        all_results_df = pd.DataFrame(columns=['categorical_column', 'p_value'])
        
        for col in columns_to_analyze:
            results_df = kruskal_h_test(df, target_col, col)
            all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
        
        kruskal_df = all_results_df.set_index('categorical_column')
        kruskal_df = kruskal_df.rename(columns={'p_value':target_col})
        combined_kruskal_test = pd.concat([combined_kruskal_test,kruskal_df],axis=1)
    
    combined_kruskal_test = combined_kruskal_test.drop(columns=['categorical_column'])

    return combined_kruskal_test

def plot_combined_kruskal_plot(combined_kruskal_test):
    threshold = 0.05
    
    # Function to calculate color intensity based on value
    def calculate_color_intensity(value, threshold):
        if value >= threshold:
            return 1  # Green
        else:
            # Calculate a scaled value between 0 (just below threshold) and -1 (for minimum value, assuming 0 as min for simplicity)
            return (value / threshold) - 1
    
    # Apply the function to normalize data
    norm_data_adjusted = combined_kruskal_test.applymap(lambda x: calculate_color_intensity(x, threshold))
    
    # Creating a new heatmap with adjusted normalization
    plt.figure(figsize=(10, 7))
    sns.heatmap(norm_data_adjusted, cmap=sns.diverging_palette(10, 133, sep=80, n=50), cbar=True,annot=True)
    
    # Customizing the plot
    plt.title("Adjusted Custom Heatmap")
    plt.yticks(rotation=0)
    plt.xticks(rotation=45, ha="right")
    
    # Show the adjusted plot
    plt.tight_layout()
    plt.show()

    return combined_kruskal_test

def top_5_category_errors(combined_kruskal_test_t,category):
    
    error_top_5_categs = pd.DataFrame()
    high_error_cols = combined_kruskal_test_t[combined_kruskal_test_t[category]<=0.05].index.tolist()
    
    for error in high_error_cols:
        top_5_vals = error_rate_table(df,error,category,category,4,3,False,order,False)
        top_5_vals = top_5_vals.reset_index(drop=False).rename(columns={category:error})
        del top_5_vals['index']
        error_top_5_categs = pd.concat([error_top_5_categs,top_5_vals],axis=1)
    return error_top_5_categs

def aggregated_months(df1,col,error_types):
    df1 = df1[error_types+[col]]
    
    aggregated_data = df1.groupby(col).sum()
    
    aggregated_data.reset_index(inplace=True)
    
    colors = ["#00798c", "#d1495b", '#edae49', '#66a182']
    
    fig, axs = plt.subplots(len(error_types), 1, figsize=(12, 8), sharex=True)
    
    
    for i, error_type in enumerate(error_types):
        axs[i].bar(aggregated_data[col], aggregated_data[error_type], color=colors[i], label=error_type)
        axs[i].set_title(error_type)
        axs[i].tick_params(labelrotation=45)
        axs[i].legend()
    
    plt.tight_layout()
    plt.show()

def outlier_counts(df,error_columns):
    
    outlier_info = {}
    
    for column in error_columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
    
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outlier_info[column] = {
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
            "outliers_count": outliers.shape[0]
        }
    
    outlier_info_df = pd.DataFrame(outlier_info).transpose()
    
    return outlier_info_df